# Bandas de Bollinger → Analise das ações utilizando as médias moveis como orientação para compra e venda.

# Bandas Bollinger
Bandas de bollinger conceitos
Quando o preço do ativo ultrapassa a banda superior, observamos uma tendência de alta do ativo. Por outro lado, se o preço fica abaixo da banda inferior, há então uma tendência de baixa.
As bandas de bollinger funcionam bem para mercados que seguem tendências. Isso mostra que, se utilizadas para analisar ativos que fogem dessa premissa, podem não ser tão úteis.

Estratégias utilizando bandas de bollinger
São utilizadas algumas estratégias baseadas nessa ferramenta de análise.  São elas:

  * Cruzamento dos preços com as bandas
  * Preço acima ou abaixo das bandas
  * Critério para saída da operação (Stop)

In [1]:
# Importando as bibliotecas
import warnings
import pandas               as pd
import yfinance             as yf
import seaborn              as sns
import plotly.graph_objects as go

In [2]:
# Configuração do padrão de visualização dos dados e graficos
# https://seaborn.pydata.org/tutorial.html
sns.set_theme(
    context='talk',
    style='ticks',
    palette='icefire',
    font_scale=.8,
    rc={'figure.figsize': (25, 15)}
)

warnings.filterwarnings("ignore")

In [3]:
# Escolhendo a ação
acao = 'PETR4.SA'

ticker = yf.Ticker(acao)

# Intervalo de tempo
df = ticker.history(period='1mo')

df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-04-19 00:00:00-03:00,24.172928,24.306431,23.541015,23.594416,73217800,0.0,0.0
2023-04-20 00:00:00-03:00,23.603316,24.012724,23.398612,23.754620,54738400,0.0,0.0
2023-04-24 00:00:00-03:00,23.674517,24.261929,23.585514,24.208529,43820900,0.0,0.0
2023-04-25 00:00:00-03:00,24.172927,24.475534,23.781318,24.110626,53854100,0.0,0.0
2023-04-26 00:00:00-03:00,24.137328,24.226328,23.808020,23.808020,40249400,0.0,0.0


In [4]:
# Calulo da MM simples e das bordas de Bollinger
#"""Centro: média móvel simples de 5 períodos (se período = dias, então: 5 dias)
#Banda superior: média móvel simples (5 períodos) + (2x desvio padrão de 5 dias)
#Banda inferior: média móvel simples (5 períodos) – (2x desvio padrão de 5 dias)"""

N = 5 # Moving Average periods 
N_ = 26
k = 2  # Factor to shift the bands

df['Standard_Deviation'] = df['Close'].rolling(N).std()
df['MM5d'] = df['Close'].rolling(N).mean()
df['MM26d'] = df['Close'].rolling(N_).mean()
df['Upper_Band'] = df['MM5d'] + df['Standard_Deviation'] * k
df['Lower_Band'] = df['MM5d'] - df['Standard_Deviation'] * k


In [7]:
#grafico
candlestick = go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']
)
lines_mm5 = go.Scatter(x=df.index,
                       y=df['MM5d'],
                       mode='lines',
                       line_color='brown',
                       name='MM5d')
lines_mm26 = go.Scatter(
                       x=df.index,
                       y=df['MM26d'],
                       mode='lines',
                       line_color='orange',
                       name='MM26d')

lines_UB = go.Scatter(
                       x=df.index,
                       y=df['Upper_Band'],
                       mode='lines',
                       line_color='red',
                       name='Upper_Band')

lines_LB = go.Scatter(
                       x=df.index,
                       y=df['Lower_Band'],
                       mode='lines',
                       line_color='green',
                       name='Lower_Band')



fig = go.Figure(data=[candlestick])
fig.update_traces(marker_autocolorscale=False, selector=dict(type='bar'))
fig.update_layout(width=1800, height=1000, font_size=16, dragmode='zoom')

fig.add_trace(lines_mm5)
fig.add_trace(lines_mm26)
fig.add_trace(lines_UB)
fig.add_trace(lines_LB)



fig.show()

## Utilizaremos como confirmação o principio do cruzamentos das 3 médias moveis exponenciais.
Ou seja, a média de 15 dias será definida como média rapida, a de 30 como média convecional e a de 100 dias como média lenta.

In [8]:
# Calculo da media movel exponencial MME

df['MME15d'] = df['Close'].ewm(span=15).mean()
df['MME30d'] = df['Close'].ewm(span=30).mean()
df['MME100d'] = df['Close'].ewm(span=100).mean()




In [9]:
#grafico
candlestick = go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']
)

lines_me15 = go.Scatter(
                       x=df.index,
                       y=df['MME15d'],
                       mode='lines',
                       line_color='brown',
                       name='MME15d')

lines_me30 = go.Scatter(
                       x=df.index,
                       y=df['MME30d'],
                       mode='lines',
                       line_color='orange',
                       name='MME30d')


lines_me100 = go.Scatter(
                       x=df.index,
                       y=df['MME100d'],
                       mode='lines',
                       line_color='pink',
                       name='MME100d')



fig = go.Figure(data=[candlestick])
fig.update_traces(marker_autocolorscale=False, selector=dict(type='bar'))
fig.update_layout(width=1800, height=700, font_size=16, dragmode='zoom')

fig.add_trace(lines_me15)
fig.add_trace(lines_me30)
fig.add_trace(lines_me100)


fig.show()